In [5]:
# Specify the path to your text file
# file_path = r'C:\Users\prs90\Downloads\tranmitter_uplane_prb_fields_frame_117.txt'
# a=[]
# b=[]
# # Function to extract iSample and qSample values from a text file
# def extract_samples(file_path):
#     i_samples = []
#     q_samples = []

#     with open(file_path, 'r') as file:
#         for line in file:
#             if "iSample:" in line:
#                 # Extract the iSample value
#                 sample_value = line.split(':')[1].strip()
#                 i_samples.append(sample_value)
#             elif "qSample:" in line:
#                 # Extract the qSample value
#                 sample_value = line.split(':')[1].strip()
#                 q_samples.append(sample_value)

#     return i_samples, q_samples
    
# # Call the function and print results
# i_samples, q_samples = extract_samples(file_path)

# for i in range(0,len(i_samples)):
#     a.append(i_samples[i][:12])
# for i in range(0,len(q_samples)):
#     b.append(q_samples[i][:12])
# # Create a new DataFrame from the list
# isa = pd.DataFrame([a])  # Use [i_samples] to create a single row

# # Rename columns to i_samples_n
# isa.columns = [f'i_samples_{i}' for i in range(len(a))]
# # Create a new DataFrame from the list
# qsa = pd.DataFrame([b])  # Use [i_samples] to create a single row

# # Rename columns to i_samples_n
# qsa.columns = [f'q_samples_{i}' for i in range(len(b))]
# final=pd.concat([isa,qsa],axis=1)
# final


In [5]:
import pyshark
import pandas as pd
import os
import nest_asyncio

# Allow multiple asyncio event loops
nest_asyncio.apply()

# Function to process the PCAP file and write the output to text files
def process_pcap_files(pcap_file, output_dir):
    # Open the PCAP file
    capture = pyshark.FileCapture(pcap_file)
    
    for packet in capture:
        # Create a unique filename for each frame
        frame_file_path = os.path.join(output_dir, f"frame_{packet.number}.txt")
        with open(frame_file_path, "w") as output_file:
            output_file.write(f"\n--- Frame {packet.number} ---\n")
            
            # O-RAN Fronthaul CUS fields
            oran_layer = getattr(packet, 'oran_fh_cus', None)
            if oran_layer:
                output_file.write(f"Oran layer: {oran_layer}\n")
                # Extract specific fields from the O-RAN Fronthaul CUS layer
                if hasattr(oran_layer, 'fields'):
                    for field in oran_layer.fields:
                        output_file.write(f"{field.name}: {field.showname_value}\n")
    print(f"PCAP processed and text files saved in {output_dir}.")

# Function to extract iSample and qSample values from a text file
def extract_samples(file_path):
    i_samples = []
    q_samples = []

    with open(file_path, 'r') as file:
        for line in file:
            if "iSample:" in line:
                # Extract the iSample value
                sample_value = line.split(':')[1].strip()
                i_samples.append(sample_value)
            elif "qSample:" in line:
                # Extract the qSample value
                sample_value = line.split(':')[1].strip()
                q_samples.append(sample_value)

    return i_samples, q_samples

# Function to process all text files and save results into a CSV
def process_text_files_and_save_csv(output_dir, csv_output_file):
    all_i_samples = []
    all_q_samples = []
    
    # Iterate through all text files in the directory
    for file_name in os.listdir(output_dir):
        if file_name.endswith(".txt"):
            file_path = os.path.join(output_dir, file_name)
            i_samples, q_samples = extract_samples(file_path)

            # Extract the first 12 characters for iSamples and qSamples
            a = [i_sample[:12] for i_sample in i_samples]
            b = [q_sample[:12] for q_sample in q_samples]

            # Append extracted values to the lists
            all_i_samples.append(a)
            all_q_samples.append(b)
    
    # Create DataFrames for i_samples and q_samples
    isa = pd.DataFrame(all_i_samples)  # Multiple rows for i_samples
    isa.columns = [f'i_samples_{i}' for i in range(isa.shape[1])]

    qsa = pd.DataFrame(all_q_samples)  # Multiple rows for q_samples
    qsa.columns = [f'q_samples_{i}' for i in range(qsa.shape[1])]

    # Concatenate i_samples and q_samples into a single DataFrame
    final_df = pd.concat([isa, qsa], axis=1)

    # Save the final DataFrame to a CSV file
    final_df.to_csv(csv_output_file, index=False)
    print(f"Data saved to {csv_output_file}")

# Main code
if __name__ == "__main__":
    # Define the path to the PCAP file
    pcap_file = r"C:\Users\Rahul Singh\Documents\pacap_project\transmitter_uplane_dl_only_5g_nr2_1cc_400MHz_TM1p1_10ms_4pcap.pcap"
    
    # Define the output directory for text files
    output_dir = r"C:\Users\Rahul Singh\Documents\pacap_project\output"
    
    # Process the PCAP file and generate text files for each frame
    process_pcap_files(pcap_file, output_dir)
    
    # Define the output CSV file
    csv_output_file = r"C:\Users\Rahul Singh\Documents\pacap_project\output\final_output.csv"
    
    # Process all text files and save the extracted iSample and qSample values to a CSV file
    process_text_files_and_save_csv(output_dir, csv_output_file)

RuntimeError: Cannot run the event loop while another loop is running